In [28]:
from selenium.webdriver.common.by import By
from time import sleep
from main import create_search_day_list, selenium_run, auto_login, to_search_date, display_smaller
import pyautogui as pg

# 抽出期間の取得
start = '2022-03-13'
end = '2022-03-13'
search_days = create_search_day_list(start, end)

# 抽出期間より1日毎取り出す
for _search_day in search_days:
    search_day = _search_day.strftime("%Y-%m-%d")

    # データ格納用
    data = []

    driver = selenium_run()
    auto_login(driver)

    # 抽出日で全件検索
    to_search_date(search_day)
    sleep(2)

    # 検索件数の取得
    search_results_count = int(driver.find_element(by=By.CLASS_NAME, value='c-tabs__tab_count').text)
    print(f'検索結果数：{search_results_count}')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\sgwya\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


検索日:2022-03-13
検索結果数：19


In [ ]:

    # 並べ替え：新 → 古
    pg.press('tab', presses=16, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)
    pg.press('down', presses=2, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)
    # 全件表示
    pg.press('tab', presses=1, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)
    pg.press('down', presses=3, interval=0.2)
    pg.press('enter', presses=1, interval=0.1)

    # 表示を小さくする
    display_smaller()

    # 一番下まで移動
    pg.press('tab', presses=3, interval=0.5)
    pg.press('pagedown', presses=10, interval=0.5)
    sleep(2)

In [19]:
message_groups = driver.find_elements(by=By.CLASS_NAME, value='c-message_group')

In [20]:
message = message_groups[12]

In [27]:
def get_text_section(message, class_name):
    text_section = message.find_element(by=By.CLASS_NAME, value=class_name).text
    print(class_name)
    try:
        # リスト表記をしているメッセージへの対応
        lists = message.find_elements(by=By.TAG_NAME, value='li')
        if lists is not None:
            for list in lists:
                text_section += list.text + '\n'
    except:
        pass
    return text_section